# Polinomios y el algoritmo de Gosper

En esta tarea vamos a trabajar con polinomions y vamos ha programar un algoritmo que calculará una forma "canónica" asociada a cualquier facción polinómica. Esta forma canónica tiene gran utilidad a la hora de estudiar series "Gosper sumables".

**Nota:** Iremos haciendo un ejemplo después de cada ejercicio para comprobar que las funciones van funcionando correctamente.

In [1]:
from sympy import *
x = symbols('x')
h = symbols('h')

# Nuestro objetivo

Vamos a trabajar con fracciones polinómicas, esto es, funciones de la forma
$$
r(x)=\frac{p(x)}{q(x)}
$$
con $p$ y $q$ polinomios.

Nuestro objetivo será obtener una forma canónica asociada a esta función
$$r(x)=cte \frac{a(x)}{b(x)} \frac{c(x+1)}{c(x)},$$
con $cte$ una constante y $a,b,c$ polinomios mónicos cumpliendo que $$m.c.d(a(x),b(x+h))=1,$$ para todo entero $h\geq 0$.

## Primer paso

Comenzamos expresando $r(x)$ como una cosntante por un cociente de polinomios mónicos 
$$
r(x)=cte \frac{p_1(x)}{q_1(x)}
$$
Si llamamos $a=$ coeficiente lider de $p(x)$ y $b=$ coeficiente lider de $q(x)$, 
claramente:
\begin{array}{l}
 cte=\frac{a}{b},\\
 p_1(x)=\frac{1}{a} p(x),\\
 q_1(x)=\frac{1}{b} q(x).
\end{array}


**Ejercicio 1.-** Define una función <span style="color:blue">paso1</span> que asocie a una función racional $r(x)$ ( o a un par de polínomios $(p(x),q(x))$ ) una terna $(cte,p_1(x),q_1(x))$ en las condiciones anteriores.

In [2]:
def paso1(p,q):
    
    #comenzamos por extraer los coeficientes líderes
    
    polyp, polyq = Poly(p), Poly(q)
    lider_p, lider_q = polyp.coeffs()[0], polyq.coeffs()[0]
    lider = lider_p / lider_q
    
    #dividimos cada polinomio por su coeficiente líder
    
    poly_p = Lambda((x),simplify(polyp/lider_p))
    poly_q = Lambda((x),simplify(polyq/lider_q))
    
    #devolvemos el cociente de los coeficientes líderes y los nuevos polinomios

    return([lider,poly_p,poly_q])    

In [3]:
p, q = 3*(x-1)**2 * (x+2), (x-1)*(x-3)

In [4]:
p1, q1 = paso1(p,q)[1], paso1(p,q)[2]
paso1(p,q)

[3, Lambda(x, x**3 - 3*x + 2), Lambda(x, x**2 - 4*x + 3)]

## Segundo paso

En el siguiente paso vamos a simplificar la fracción $\frac{p_1(x)}{q_1(x)}$ para ello calculamos $m(x)=m.c.d(p_1,q_1)$ y tomamos $p_2(x)=\frac{p_1(x)}{m(x)}$ y $q_2(x)=\frac{q_1(x)}{m(x)}$

** Ejercicio 2.-**  Define una función <span style="color:blue">paso2</span> que asocie a un par de polinomios $(p_1,q_1)$ un par de polinomios $(p_2,q_2)$ en las condiciones anteriores.

In [5]:
def paso2(p,q):
    
    polyp, polyq = Poly(p(x)), Poly(q(x))
    m = gcd(polyp,polyq)
    
    #calculmos el mcd y devolvemos los polinomios divididos por éste
    
    return([Lambda((x),simplify(polyp/m)),Lambda((x),simplify(polyq/m))])

In [6]:
p2, q2 = paso2(p1,q1)[0], paso2(p1,q1)[1]
paso2(p1,q1)

[Lambda(x, x**2 + x - 2), Lambda(x, x - 3)]

## Tercer paso

Recordar que nuestro objetivo es expresar una función racional $r$ en forma canónica:
$$r(x)=cte \frac{a(x)}{b(x)} \frac{c(x+1)}{c(x)},$$
con la condición $m.c.d(a(x),b(x+h))=1,$ para todo $h$ entero positivo.

### Definición de la función paso3

** Ejercicio 3.-** Definir la función <span style="color:blue">paso3</span> con las siguientes entradas y salidas:
- **Entrada:**
   - Dos polinomios $p$ y $q$ mónicos y primos relativos.
   
- **Salida:**
   - Polinomios $a(x), b(x), c(x)$ tales que:
   $$\qquad$$
      - $\frac{p(x)}{q(x)}=\frac{a(x)}{b(x)} \frac{c(x+1)}{c(x)}$,
      $$\qquad$$
      - $m.c.d(a(x),b(x+h))=1$, para todo entero $h\geq 0$.

Seguiremos el siguiente algoritmo:

- **Paso 3.1.** Calculamos la resultante

$$R(h)=resultant(p(x), q(x+ h),x)$$

- **Paso 3.2.** Encontramos la lista de raíces enteras no negativas de $R(h)$ y las ordenamos de menor a mayor

$$
ListaRaices=\{0\leq  h_1\leq h_2\leq \ldots \leq h_N\}
$$

- **Paso 3.3.** Tomamos 
$$\qquad$$
    - $p_0(x)= p(x)$, 
     
    - $q_0(x)=q(x)$. 
    $$\qquad$$

- **Paso 3.4.** Para $j=1, \ldots, N$ tomamos
\begin{array}{l}
    s_j(x)= mcd(p_{j-1}(x),q_{j-1}(x+h_j)), \\
    p_j(x)= \frac{p_{j-1}(x)}{s_j(x)}, \\
    q_j(x)= \frac{q_{j-1}(x)}{s_j(x-h_j)}.
\end{array}
    
- **Paso 3.5.** Salida
\begin{array}{l}
a(x)= p_N(x),\\
b(x)= q_N(x),\\
c(x)= \prod_{i=1}^N \prod_{j=1}^{h_i} s_i(x-j).
\end{array}


In [7]:
#función que nos calcula las raíces que cumplen la condición del enunciado

def raices(p,q):
    
    polyp, polyq = p, q
    listaraices = [x for x in solve(resultant(polyp(x),polyq(x+h),x),h) if ask(Q.integer(x)) and x > 0]
    
    return(listaraices)

In [8]:
def paso3(p,q):
    
    p0, q0 = p, q
    listaraices = raices(p,q)
    n = len(listaraices)
    s_lista = []
    
    #bucle que calcula las funciones p, q y s
    
    for i in range(n):
        s_lista.append(Lambda((x),gcd(p0(x),q0(x+listaraices[i]))))
        p0, q0 = simplify(Lambda((x), p0(x)/s_lista[i](x))), simplify(Lambda((x),q0(x)/s_lista[i](x-listaraices[i])))
        
    c = 1
    
    #bucle que calcula la función c
    
    for i in range(n):
        for j in range(1,listaraices[i]+1):
            c = c*s_lista[i](x-j)
        
    a = simplify(p0)
    b = simplify(q0)
    c = Lambda((x),c)
    
    return([a,b,c])        

In [9]:
a, b, c = paso3(p2,q2)
paso3(p2,q2)

[Lambda(x, x + 2), Lambda(x, 1), Lambda(x, (x - 3)*(x - 2))]

In [10]:
solve(resultant(a(x),b(x+h),x),h)

[]

**Ejercicio 4.-** Usas las funciones <span style="color:blue">paso1,paso2</span> y <span style="color:blue">paso3</span> para definer una función <span style="color:blue">Gosper</span> con entrada una función racional $r(x)$ (o un par de polinomios $(p(x),q(x))$ ) y salida una cuatrupla $(cte,a(x),b(x),c(x))$ correspondiente a la expresión en forma canónica de $r(x)$.

In [11]:
def gosper(p,q):
    step1 = paso1(p,q)
    step2 = paso2(step1[1],step1[2])
    step3 = paso3(step2[0],step2[1])
    return([step1[0],step3[0],step3[1],step3[2]])

In [12]:
gosper(p,q)

[3, Lambda(x, x + 2), Lambda(x, 1), Lambda(x, (x - 3)*(x - 2))]

In [13]:
lider, a, b, c = gosper(p,q)
solve(resultant(a(x),b(x+h),x),h)

[]

**Ejercicio 5.** Aplica la función <span style="color:blue">Gosper</span> a las funciones racionales determinadas por los siguientes pares de polinomios:


\begin{array}{l}
(p=2x^4 - 22x^3 + 48x^2 + 46x + 70\, , q=5x^6 - 75x^5 + 315x^4 - 320 x^3 + 385x^2 - 550 x + 240),\\
(p=3 x^5 - 15x^4 + 33x^2 + 51 x + 36\, , q=5 x^7 - 65 x^6 + 165 x^5 + 300 x^4 - 100x^3 - 475 x^2 - 70 x + 240)\\
( p=7x^7 - 49x^6 - 245x^5 + 1330x^4 + 1813x^3 - 2681x^2 - 1575 x + 1400 \, , q=2x^5 - 36 x^3 + 60 x^2 - 38 x + 60).
\end{array}

Comprueba que se da la igualdad 
$$\frac{p(k)}{q(k)}=cte \frac{a(k)}{b(k)} \frac{c(k+1)}{c(k)}$$
para cada pareja $(p,q)$. 

Comprueba también que los polinomios que has obtenido $a(k)$ y $b(k+h)$ son primos relativos, para todo entero no negativo $h$. Para comprobar esto último calcula la correspondiente resultante y comprueba que no tiene raíces enteras no negativas.

### Ejemplo1

In [14]:
p = 2*x**4 - 22*x**3 + 48*x**2 + 46*x + 70
q = 5*x**6 - 75*x**5 + 315*x**4 - 320*x**3 + 385*x**2 - 550*x + 240
gosper(p,q)

[2/5,
 Lambda(x, x**2 + x + 1),
 Lambda(x, x**4 - x**3 + x**2 - 2*x + 1),
 Lambda(x, -12*x + (x - 1)**2 + 47)]

Pasamos a hacer la comprobación pertinente:

In [15]:
a = Lambda((x), x**2 + x + 1)
b = Lambda((x), x**4 - x**3 + x**2 - 2*x + 1)
solve(resultant(a(x),b(x+h),x),h)

[3/2 - sqrt(3)*I/2,
 3/2 + sqrt(3)*I/2,
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 0),
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 1),
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 2),
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 3),
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 4),
 CRootOf(h**6 - 3*h**5 + 8*h**4 - 13*h**3 + 13*h**2 - 4*h + 1, 5)]

### Ejemplo 2

In [16]:
p = 3*x**5 - 15*x**4 + 33*x**2 + 51*x + 36
q = 5*x**7 - 65*x**6 + 165*x**5 + 300*x**4 - 100*x**3 - 475*x**2 - 70*x + 240
gosper(p,q)

[3/5,
 Lambda(x, x**2 + x + 1),
 Lambda(x, x**4 - x**2 - x + 1),
 Lambda(x, (x - 8)*(x - 7)*(x - 6)**2*(x - 5)**2*(x - 4))]

Pasamos a hacer la comprobación pertinente:

In [17]:
a = Lambda((x), x**2 + x + 1)
b = Lambda((x), x**4 - x**2 - x + 1)
solve(resultant(a(x),b(x+h),x),h)

[3/2 - sqrt(3)*I/2,
 3/2 + sqrt(3)*I/2,
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 0),
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 1),
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 2),
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 3),
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 4),
 CRootOf(h**6 - h**5 + 2*h**4 - 3*h**3 + 3*h**2 + 4*h + 1, 5)]

### Ejemplo 3

In [18]:
p = 7*x**7 - 49*x**6 - 245*x**5 + 1330*x**4 + 1813*x**3 - 2681*x**2 - 1575*x + 1400
q = 2*x**5 - 36*x**3 + 60*x**2 - 38*x + 60
gosper(p,q)

[7/2,
 Lambda(x, x**4 - 12*x**3 + 26*x**2 + 53*x - 40),
 Lambda(x, x**2 + 1),
 Lambda(x, x*(x - 3)*(x - 2)**2*(x - 1))]

Pasamos a hacer la comprobación pertinente:

In [19]:
a = Lambda((x), x**4 - 12*x**3 + 26*x**2 + 53*x - 40)
b = Lambda((x), x**2 + 1)
solve(resultant(a(x),b(x+h),x),h)

[-8 - I,
 -8 + I,
 -5 - I,
 -5 + I,
 1/2 - sqrt(1/4 - sqrt(5)*I),
 1/2 + sqrt(1/4 - sqrt(5)*I),
 1/2 - sqrt(1/4 + sqrt(5)*I),
 1/2 + sqrt(1/4 + sqrt(5)*I)]

### Conclusión

En ninguno de los ejemplos la resultante tiene una raíz entera no negativa.